In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("adult.data.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


#### How many people of each race are represented in this dataset? This should be a Pandas series with race names as the index labels. (`race` column)

In [4]:
race_count = df.groupby("race").race.count().sort_values(ascending=False)
race_count

race
White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

#### What is the average age of men?

In [5]:
average_age_men = round(df.groupby("sex").age.mean().loc["Male"], 1)
average_age_men

39.4

#### What is the percentage of people who have a Bachelor's degree?

In [6]:
percentage_bachelors = round(df.groupby("education").education.count().loc["Bachelors"] / len(df) * 100, 1)
percentage_bachelors

16.4

#### What percentage of people with advanced education (`Bachelors`, `Masters`, or `Doctorate`) make more than 50K?

In [7]:
mask_hi_ed = df.education.isin(["Bachelors", "Masters", "Doctorate"])

In [8]:
hi_ed = df.loc[mask_hi_ed].groupby(["salary", "education"]).salary.count()
hi_ed

salary  education
<=50K   Bachelors    3134
        Doctorate     107
        Masters       764
>50K    Bachelors    2221
        Doctorate     306
        Masters       959
Name: salary, dtype: int64

In [9]:
hi_ed_all = hi_ed.sum()
hi_ed_all

7491

In [10]:
hi_ed_m_50 = hi_ed.loc[">50K"].sum()
hi_ed_m_50

3486

In [11]:
higher_education_rich = round(hi_ed_m_50 / hi_ed_all * 100, 1)
higher_education_rich

46.5

#### What percentage of people without advanced education make more than 50K?

In [12]:
lo_ed = df.loc[~mask_hi_ed].groupby(["salary", "education"]).salary.count()
lo_ed

salary  education   
<=50K   10th             871
        11th            1115
        12th             400
        1st-4th          162
        5th-6th          317
        7th-8th          606
        9th              487
        Assoc-acdm       802
        Assoc-voc       1021
        HS-grad         8826
        Preschool         51
        Prof-school      153
        Some-college    5904
>50K    10th              62
        11th              60
        12th              33
        1st-4th            6
        5th-6th           16
        7th-8th           40
        9th               27
        Assoc-acdm       265
        Assoc-voc        361
        HS-grad         1675
        Prof-school      423
        Some-college    1387
Name: salary, dtype: int64

In [13]:
lo_ed_all = lo_ed.sum()
lo_ed_all

25070

In [14]:
lo_ed_m_50 = lo_ed.loc[">50K"].sum()
lo_ed_m_50

4355

In [15]:
lower_education_rich = round(lo_ed_m_50 / lo_ed_all * 100, 1)
lower_education_rich

17.4

#### What is the minimum number of hours a person works per week?

In [16]:
min_work_hours = df.loc[:, "hours-per-week"].min()
min_work_hours

1

#### What percentage of the people who work the minimum number of hours per week have a salary of more than 50K?

In [17]:
mask_min_hours = df["hours-per-week"] == min_work_hours

In [18]:
min_hours_sal = df.loc[mask_min_hours].groupby("salary").salary.count()
min_hours_sal

salary
<=50K    18
>50K      2
Name: salary, dtype: int64

In [19]:
rich_percentage = round(min_hours_sal.loc[">50K"] / min_hours_sal.sum() * 100, 1)
rich_percentage

10.0

#### What country has the highest percentage of people that earn >50K and what is that percentage?

In [20]:
country_salary = df.groupby(["native-country", "salary"]).salary.count().unstack()
country_salary.head()

salary,<=50K,>50K
native-country,,
?,437.0,146.0
Cambodia,12.0,7.0
Canada,82.0,39.0
China,55.0,20.0
Columbia,57.0,2.0


In [21]:
country_salary["rich_perc"] = country_salary.loc[:, ">50K"].div(country_salary.sum(axis=1)).mul(100)
country_salary.head()

salary,<=50K,>50K,rich_perc
native-country,,,
?,437.0,146.0,25.042882
Cambodia,12.0,7.0,36.842105
Canada,82.0,39.0,32.231405
China,55.0,20.0,26.666667
Columbia,57.0,2.0,3.389831


In [22]:
highest_earning_country = country_salary.rich_perc.idxmax()
highest_earning_country

'Iran'

In [23]:
highest_earning_country_percentage = round(country_salary.rich_perc.max(), 1)
highest_earning_country_percentage

41.9

####  Identify the most popular occupation for those who earn >50K in India. 

In [24]:
mask_country = df["native-country"]=="India"

In [25]:
top_IN_occupation = df.loc[mask_country].groupby(["salary", "occupation"]).salary.count().loc[">50K"].idxmax()
top_IN_occupation

'Prof-specialty'